# Relatório da disciplina de Algebra Linear Computacional (CKP8122)
 - Aluno: Madson Luiz Dantas Dias
 - Professor: Creto Augusto Vidal

## Lista de implementações
### básico
 1. Classe Matriz
 2. Multiplicação de vetor-matriz e matriz-vetor

### métodos diretos
 3. eliminação de Gauss
 4. decomposição LU
 5. decomposição de Cholesky

In [1]:
from Matrix import *

## Básico
### Classe Matriz
#### Código Python
```Python
import math
import copy
from random import random

# ones method
def ones(dimensions):
    return (rand(dimensions) * 0) + 1

# zeros method
def zeros(dimensions):
    return rand(dimensions) * 0

# rand method
def rand(dimensions):
    if type(dimensions) not in [int, tuple]:
        print("Error: inconsistent dimensions.")
        return False
    if type(dimensions) == int:
        dimensions = (dimensions, dimensions)
    return Matrix([[random() for i in range(dimensions[1])] for j in range(dimensions[0])])

# eye method
def eye(dimension):
    if type(dimension) is int:
        result = zeros(dimension)
        for i in range(result.shape[0]): result[i,i] = 1
        return result
    else:
        print("error: dimension not int")
        return False

# Matrix class
class Matrix(object):
    def __init__(self,linhas = list()):
        self.linhas = linhas
        if type(linhas) == list:
            if len(set([len(i) for i in linhas])) > 1:
                print("Error: matrix not created, size of rows is inconsistent.")
            else:
                if len(linhas) == 0:
                    self.shape = (0,0)
                else:
                    self.shape = (len(linhas),len(linhas[0]))
        else:
            print("Error: Data structure must contain a list of lists (eg: [[1,2],[3,4]]).")
    
    def __str__(self):
        s = '\n'
        for linha in self.linhas:
            ss = ''
            for coluna in linha:
                ss = ss + ' %7.2f ' % coluna
            s =  s + '|' + ss + '|\n'
        return s

    def __item2list__(self, i,j):
        list_index = [0,0]
        for dim,count in [(i,0),(j,1)]:
            if type(dim) is int:
                list_index[count] = [self.shape[count] + dim if dim < 0 else dim]

            if type(dim) is slice:
                start = 0 if dim.start == None else dim.start
                step  = 1 if dim.step == None else dim.step
                stop = self.shape[count] if dim.stop == None else dim.stop
                # step = -1 if start > stop and step == 1 else step

                if stop > start and step < 0: stop, start = start, stop-1
                stop = self.shape[count] + stop if stop < 0 else stop

                list_index[count] = list(range(start, stop,step))

            if type(dim) is Matrix:
                dim = dim.transpose().linhas[0] if dim.shape[0] > dim.shape[1] else dim.linhas[0]
            if type(dim) is set: dim = list(dim)
            if type(dim) is list: list_index[count] = [i.linhas[0] if type(i) is Matrix else i for i in dim]
        return list_index

    def __getitem__(self, item):
        i,j = item
        if type(i) is int and type(j) is int:
            return self.linhas[i][j]

        list_index = self.__item2list__(i,j)

        result = list()
        for row in list_index[0]:
            aux_list = list()
            for col in list_index[1]:
                aux_list.append(self.linhas[row][col])
            result.append(aux_list)
        return Matrix(result)
            
    def __setitem__(self, item, value):
        i, j = item
        if type(i) is int and type(j) is int:
            self.linhas[i][j] = value
        else:
            list_index = self.__item2list__(i, j)
            if type(value) is int:
                value = ones((len(list_index[0]), len(list_index[1]))) * value
            row_ = 0
            for row in list_index[0]:
                col_ = 0
                for col in list_index[1]:
                    self[row,col] = value[row_,col_]
                    col_ += 1
                row_ += 1

    def __add__(self, value):
        if type(value) == Matrix:
            if self.shape != value.shape:
                print("Error: incompatible dimensions.")
                return False
            else:
                result = copy.deepcopy(self)
                for i in range(self.shape[0]):
                    for j in range(self.shape[1]):
                        result[i,j] = result[i,j] + value[i,j]
        elif type(value) in [int,float]:
            result = copy.deepcopy(self) + Matrix([[value]*self.shape[1]]*self.shape[0])
        return result

    def __sub__(self, value):
        if type(value) == Matrix:
            if self.shape != value.shape:
                print("Error: incompatible dimensions.")
                return False
            else:
                result = copy.deepcopy(self)
                for i in range(self.shape[0]):
                    for j in range(self.shape[1]):
                        result[i,j] = result[i,j] - value[i,j]
        elif type(value) in [int,float]:
            result = copy.deepcopy(self) - Matrix([[value]*self.shape[1]]*self.shape[0])
        return result

    def __neg__(self): return self * (-1)

    def __mul__(self, value):
        result = copy.deepcopy(self)
        if type(value) in [int, float]:
            for i in range(self.shape[0]):
                for j in range(self.shape[1]):
                    result[i,j] = result[i,j] * value
        elif type(value) == Matrix and self.shape[1] != value.shape[0]:
            print("Error: Matrices with Incompatible Dimensions.")
            result = None
        else:
            result = zeros((self.shape[0], value.shape[1]))
            if self.shape[0] == 1 and value.shape[1] == 1:
                result = 0
                for i in range(self.shape[1]): result = result + self[0,i] * value[i,0]
            else:
                result = zeros((self.shape[0], value.shape[1]))
                for i in range(self.shape[0]):
                    for j in range(value.shape[1]):
                        result[i,j] = self[i,:] * value[:,j]
        return result

    def __abs__(self): return self.__dot_operation__(abs)

    def transpose(self):
        result = zeros((self.shape[1], self.shape[0]))
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                result[j,i] = self[i,j]
        return result

    def __operation__(self, operation, axis=0, arg=0):
        operations_list = {'sum': sum, 'min': min, 'max': max}
        result = list()
        if axis == 1:
            for i in range(self.shape[0]):
                if arg == 0: result.append([operations_list[operation](self.linhas[i])])
                else: result.append([self.linhas[i].index(operations_list[operation](self.linhas[i]))])
        else:
            if arg == 0: return self.transpose().__operation__(operation, axis=1).transpose()
            else: return self.transpose().__operation__(operation, axis=1, arg=1).transpose()
        return Matrix(result)

    def copy(self): return copy.deepcopy(self)

    def sum(self, axis=0): return self.__operation__('sum', axis=axis)
    def max(self, axis=0): return self.__operation__('max', axis=axis)
    def argmax(self, axis=0): return self.__operation__('max', axis=axis, arg=1)
    def min(self, axis=0): return self.__operation__('min', axis=axis)
    def argmin(self, axis=0): return self.__operation__('min', axis=axis, arg=1)

    def norm(self):
        if self.shape[1] == 1:
            return pow(self.transpose() * self, 1/2)
        else:
            return pow(self * self.transpose(), 1/2)

    def dot(self, value):
        result = zeros(self.shape)
        if type(value) == Matrix and self.shape == value.shape:
            for i in range(self.shape[0]):
                for j in range(self.shape[1]):
                    result[i,j] = self[i,j] * value[i,j]
        else:
            print("Error: types or dimensions incompatibles.")
            result = None
        return result

    def concat(self, value, axis=1):
        if axis == 1 and self.shape[0] == value.shape[0]:
            result = zeros((self.shape[0], self.shape[1] + value.shape[1]))
            result[:,0:self.shape[1]] = copy.deepcopy(self)
            result[:,self.shape[1]:]  = value
        elif axis == 0 and self.shape[1] == value.shape[1]:
            result = zeros((self.shape[0] + value.shape[0], self.shape[1]))
            result[:self.shape[0],:] = copy.deepcopy(self)
            result[self.shape[0]:,:] = value
        else:
            print("Error: matrices with incompatible dimensions.")
            result = None
        return result
    
    def __dot_operation__(self, operation):
        result = zeros(self.shape)
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                result[i,j] = operation(self[i,j])
        return result

    def power(self, n): return self.__dot_operation__(lambda x: pow(x,n))

    def to_number(self):
        if self.shape == (1,1):
            return self.linhas[0][0]

    def trace(self):
        return sum([self[i,i] for i in range(self.shape[0])]) if self.shape[0] == self.shape[1] else False
    
    def forward_substituition(self):
        N, M = self.shape
        # vector of soluctions
        x = zeros((1,N))

        # auxiliary matrices
        A = self[:,:-1]
        b = self[:,-1]
        x[0,0] = b[0,0] / A[0,0]
        for i in range(1,N):
            list_index = list(range(0,i))
            x[0,i] = (-A[i,list_index].dot(x[0,list_index]).sum(axis=1) + b[i,0]) * (1/A[i,i])
            x[0,i] = x[0,i].to_number() if type(x[0,i]) is Matrix else x[0,i]
        return x    
    def back_substituition(self):
        N, M = self.shape
        # vector of soluctions
        x = zeros((1,N))

        # auxiliary matrices
        A = self[:,:-1]
        b = self[:,-1]

        if sum([A[i,i] == 0 for i in range(N)]) == 0:
            x[0,N-1] = b[N-1,0] / A[N-1,N-1]
            for j in range(N-2,-1,-1):
                x[0,j] = (b[j,0] - A[j,j+1:].dot(x[0,j+1:]).sum(axis=1).to_number() ) / A[j,j]
            return x
        else:
            print("the system not can solved by back substituition.")

```

### Multiplicação  de vetor-matriz e matriz-vetor

#### Definição
Dada duas matrizes $\mathbf{A}\in\mathcal{R}^{m\times n}$ e $\mathbf{B}\in\mathcal{R}^{n\times p}$, a multiplicação de $\mathbf{A}$ por $\mathbf{B}$ é representada por outra matriz $\mathbf{C} = \mathbf{A}\mathbf{B} \in \mathcal{R}^{m \times p}$, em que cada elemento é dado pela seguinte equação

\begin{equation}
    c_{ij} = \sum_{k=1}^{n} a_{ik}b_{kj}, ~~ i = 1,2,\dots,m ~~ e ~~ j=1,2,\dots,p
\end{equation}


#### Código python
```Python
def __mul__(self, value):
    result = copy.deepcopy(self)
    if type(value) in [int, float]:
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                result[i,j] = result[i,j] * value
    elif type(value) == Matrix and self.shape[1] != value.shape[0]:
        print("Error: Matrices with Incompatible Dimensions.")
        result = None
    else:
        result = zeros((self.shape[0], value.shape[1]))
        if self.shape[0] == 1 and value.shape[1] == 1:
            result = 0
            for i in range(self.shape[1]): result = result + self[0,i] * value[i,0]
        else:
            result = zeros((self.shape[0], value.shape[1]))
            for i in range(self.shape[0]):
                for j in range(value.shape[1]):
                    result[i,j] = self[i,:] * value[:,j]
    return result
```

In [2]:
A = Matrix([[1,2,3],[4,5,6],[7,8,9]])
B = Matrix([[10,6,9],[-1,7,9],[4,6,1]])
print(A)
print('*')
print(B)
print('=')
print(A*B)


|    1.00     2.00     3.00 |
|    4.00     5.00     6.00 |
|    7.00     8.00     9.00 |

*

|   10.00     6.00     9.00 |
|   -1.00     7.00     9.00 |
|    4.00     6.00     1.00 |

=

|   20.00    38.00    30.00 |
|   59.00    95.00    87.00 |
|   98.00   152.00   144.00 |



## Métodos diretos
### Eliminação de Gauss

#### Definição
Consideremos o sistema $\mathbf{A}\boldsymbol{x}=\boldsymbol{b}$ em que $\mathbf{A}$ é uma matriz quadrada $n \times n$.

O objetivo do método de eliminação de Gauss é eliminar os elementos $a_{ij}$ de forma a obter um sistema equivalente com uma matriz triangular superior. Depois bastará usar substituições sucessivas para chegarmos à solução pretendida. Este método pode ser sumarizado nas seguintes etapas:

 1. obtenção da matriz aumentada $[\mathbf{A} | \boldsymbol{b}]$ do sistema
 2. transformação da matriz aumentada $[\mathbf{A} | \boldsymbol{b}]$ em $[\bar{\mathbf{A}} | \bar{\boldsymbol{b}}]$, onde $\bar{\mathbf{A}}$ é uma matriz triangular superior.
 3. resolver o sistema linear $[\bar{\mathbf{A}} | \bar{\boldsymbol{b}}]$ usando retro substituição
 

O processo de transformação na matriz aumentada é baseado no pivoteamento da matriz de coeficientes $\mathbf{A}$. Primeiro, defini-se um pivô $a_{ii}$ e aplica-se as seguintes operações:
\begin{equation}
    \begin{split}
        &1.\quad L_{i} = \frac{L_{i:}}{a_{ii}},\\
        &2.\quad L_{k} = L_{k} + \alpha L_{i},\quad\forall k = i+1,\dots,n,\:\:\alpha = -\frac{a_{ki}}{a_{ii}}\\
    \end{split}
\end{equation}
em que $L_{i}$ é a $i$-ésima linha da matriz composta por $[\mathbf{A}|\boldsymbol{b}]$. Este processo é realizado para todas as linhas de $[\mathbf{A}|\boldsymbol{b}]$.

Depois de terminado o processo de redução a diante, o processo de retro-substituição é realizado da seguinte forma:
\begin{equation}
    x_i = b_i - \sum_{j=n}^{i+1} a_{ij}x_j,\quad\forall i=\dots,n.
\end{equation}

O algoritmo em python para o processo de eliminação de Gauss segue abaixo.
 
#### Código Python
##### Eliminação de Gauss
```Python
def gauss_elimination(self, b):
    Ab_ = self.concat(b)
    N, M = Ab_.shape
    # phase 1: convert A to a superior triangular matrix
    for n in range(N):
        pivot = Ab_[n,n]
        for i in range(n+1,N):
            f = Ab_[i,n] / pivot
            Ab_[i,:] = Ab_[i,:] - (Ab_[n,:] * f)
    # phase 2: solve the system using back substituition
    return Ab_.back_substituition()

```
##### Retro substituição
```Python
def back_substituition(self):
    N, M = self.shape
    # vector of soluctions
    x = zeros((1,N))

    # auxiliary matrices
    A = self[:,:-1]
    b = self[:,-1]

    if sum([A[i,i] == 0 for i in range(N)]) == 0:
        x[0,N-1] = b[N-1,0] / A[N-1,N-1]
        for j in range(N-2,-1,-1):
            x[0,j] = (b[j,0] - A[j,j+1:].dot(x[0,j+1:]).sum(axis=1).to_number() ) / A[j,j]
        return x
    else:
        print("the system not can solved by back substituition.")
        return False
```

In [3]:
A = Matrix([[ 1,-1, 2],
            [ 2, 1,-1],
            [-2,-5, 3]])
b = Matrix([[ 2],
            [ 1],
            [ 3]])

x = A.gauss_elimination(b)

print("x = ")
print(x)

x = 

|    1.00    -1.00    -0.00 |



### Decomposição LU
#### Definição
O objetivo da decomposição LU é decompor uma determinada matriz de coeficientes $\mathbf{A}\in\mathcal{R}^{n\times n}$ como o produto $\mathbf{L}\mathbf{U}$ de uma matriz $\mathbf{L}\in\mathcal{R}^{n\times n}$ triangular inferior e uma matriz $\mathbf{U}\in\mathcal{R}^{n\times n}$ triangular superior, tal que

\begin{eqnarray}
\mathbf{A}\boldsymbol{x} &=& \boldsymbol{b} \\
(\mathbf{L}\mathbf{U})\boldsymbol{x} &=& \boldsymbol{b} \\
\mathbf{L}(\mathbf{U}\boldsymbol{x}) &=&\boldsymbol{b} \\
\mathbf{L} \boldsymbol{y} = \boldsymbol{b} \quad & \text{ e }& \quad \mathbf{U}\boldsymbol{x}=\boldsymbol{y}
\end{eqnarray}

Teoricamente, a fatoração LU é equivalente ao método da eliminação de Gauss. Na prática, na fatoração LU guardamos os multiplicadores usados para transformar $\mathbf{A}$ numa matriz triangular superior $\mathbf{U}$.

#### Código Python
```Python
def lu_decomposition(self):
    N = self.shape[0]
    L = eye(N)
    U = copy.deepcopy(self)
    for n in range(N):
        L[n+1:,n] = U[n+1:,n] * (1/U[n,n])
        for l  in range(n+1,N):
            U[l,:] = U[l,:] - (U[n,:] * L[l,n])
    return L, U
```

In [2]:
L, U = A.lu_decomposition()

y = L.concat(b,axis=1).forward_substituition().transpose()

x = U.concat(y,axis=1).back_substituition()

print(x)


|    1.00    -1.00    -0.00 |



### Decomposição de Cholesky

#### Definição

O objetivo da decomposição de Cholesky é decompor uma determinada matriz definida positiva de coeficientes $\mathbf{A}\in\mathcal{R}^{n\times n}$ como o produto $\mathbf{L}\mathbf{L}^T$ de uma matriz $\mathbf{L}\in\mathcal{R}^{n\times n}$  triangular inferior com elementos da diagonal principal estritamente positivos, tal que

\begin{eqnarray}
l_{kk} &=& \sqrt{a_{kk} - \sum_{i=1}^{k-1} l_{ki}^2 }, \\
l_{kj} &=& \frac{a_{jk} - \sum_{i=1}^{k-1} l_{ji} l_{ki}}{l_{kk}}, ~~ j=k+1,k+2\dots, n
\end{eqnarray}

#### Código Python
```Python
def chol_decomposition(self):
    N = self.shape[0]
    L = zeros(N)
    for i in range(N):
        L[i,i] = pow(- (L[i,:] * L[i,:].transpose()) + self[i,i], 1/2)
        for j in range(i+1,N):
            L[j,i] = (- (L[i,:] * L[j,:].transpose()) + self[j,i]) / L[i,i]
    return L
```

In [9]:
A = Matrix([[ 4, 2,-4],
            [ 2,10, 4],
            [-4, 4, 9]])

L = A.chol_decomposition()


y = L.concat(b,axis=1).forward_substituition().transpose()

x = L.transpose().concat(y,axis=1).back_substituition()

print(x)

x = A.gauss_elimination(b)
print(x)


|    7.17    -3.33     5.00 |


|    7.17    -3.33     5.00 |

